# OpenStreetMap Data Wrangling and Case Study
***
***

## Map Area
The data covers all of lower Manhattan (south of about 40th Street) and parts of Brooklyn including portions of DUMBO, Vinegar Hill, Navy Yard, and Williamsburg.  The area is between 40.7562 and 40.6976 latitude and -74.0199 and -73.9661 longitude.

<img src = "map_area.png">

In [2]:
%load_ext sql
%sql sqlite:///osm.db

'Connected: None@osm.db'

There are three types of entities in the map: nodes, ways, and relations.  In addition, nodes have tags, ways have tags and nodes, and relations have members and tags.  Each of these are represented in a separate table.  The vast majority of the interesting data lies within the tags.

In [2]:
%%sql
SELECT name
FROM sqlite_master
WHERE type='table'
ORDER BY name;

Done.


name
Node
NodeTag
Relation
RelationMember
RelationTag
Way
WayNode
WayTag


In [ ]:
# Get the most frequent users, repeat users
# Get additions by month

In [ ]:
%%sql
CREATE VIEW tags AS
SELECT * FROM NodeTag 
UNION ALL
SELECT * FROM WayTag
UNION ALL
SELECT * FROM RelationTag;

## Clean Tag Fields
Investigated key fields that were found to be clean:
* addr:district
* addr:door
* addr:interpolation
* addr:suburb
* addr:suite
* building:colour
* delivery
* maxspeed
* oneway
* roof:colour
* roof:shape

The values for these keys were found to belong to the list of acceptable values as defined by the OSM Wiki.

In [13]:
%%sql
SELECT value AS 'Maximum Speed', COUNT(*) AS Count 
FROM tags
WHERE key='maxspeed'
GROUP BY value
ORDER BY Count DESC
LIMIT 20;

Done.


Maximum Speed,Count
25 mph,1624
35 mph,113
20 mph,66
40 mph,41
30 mph,30
45 mph,25
5 mph,5


## Problems Encountered in the Map Data

Fields with known problems:
* zip code (addr:postcode, postal_code)
* street (addr:street)
* city (addr:city)
* state (addr:state)
* country (addr:country)
* phone number (phone, contact:phone, phone:emergency, contact:fax, Phone, phone_1, fax)

Fields cleaned with simple mapping either of only values:
* roof:material
* building:material
* height
* min_height
* nycdoitt:bin
* building:part
* building:levels
* highway
* addr:floor

Fields cleaned with simple mapping either of only keys:
* leisure

Fields cleaned with mapping keys and values:
* addr:place
* capacity



### Zip Codes
Postcodes were represented inconsistently in the data for the 'addr:postcode'.  Some included a prefix of NY or were in the ZIP+4 format.  These zip codes were fixed programmatically using regular expressions.

I also manually assembled a list of valid zip codes for the areas included in the map.  Some postcodes in the data set were not present in the list.  For instance, 10023, which represents the Lincoln Square area of Manhattan, was erroneously assigned to an address near City Hall.  These zip codes were manually corrected with a dictionary mapping.  Typos in postcodes, such as ‘100014’ which has six digits, were also corrected manually.

Example errors in zip codes:
* 'NY 10007'
* '10002-1013'
* '100014'
* '10023'

The postcodes in the 'postal_code' were clean, but needed to have their key changed to 'addr:postcode'.

In [4]:
%%sql
SELECT value AS 'Zip Code', COUNT(*) AS Count 
FROM tags
WHERE key='addr:postcode'
GROUP BY value
ORDER BY Count DESC
LIMIT 10;

Done.


Zip Code,Count
10011,2824
10003,2563
10014,2556
10002,2494
10013,2175
10016,1925
10009,1685
10001,1666
10012,1570
10010,971


### Address Unit
There were only a few values that were non-numeric and hence not valid values for the key of 'addr:unit'.  For these values, if value contained 'Suite' then the key was changed to 'addr:suite' and the number was extracted and assigned to the value (similar for those that contain floor).  For all other values, the number was extracted and emitted.  In all cases, regular expressions were used.

Example errors in 'addr:unit':
* 'Suite #1'
* '19th Floor'
* 'Suite 8500'
* '#37'

### Street
Out of the fields I investigated, 'addr:street' had the most problems.  There were problems with, and including examples:
* abbreviations of street types (e.g. St., Ave)
* no street types (e.g. Old Fulton)
* abbreviations of directions (e.g. S., E)
* inconsistency in numbered streets (Second Avenue vs. 2nd Avenue)
* including housenumbers
* including suite numbers
* including city
* including state
* including floor
* including zip code
* inconsistent capitalization (Avenue Of The Americas vs. Avenue of the Americas)
* wrong or no contraction on ordinal numbers (12nd Street, 31 Street)
* unnecessary appostrophes (e.g. Saint Mark's Place)
* wrong or inconsistent spacing (Laguardia Place vs. La Guardia Place)
* not valid streets (e.g. Gramery Park, World Trade Center)


Example errors in addr:street:
* '109 East 42nd Street'
* '10th Ave #2E'
* '230th East 51st Street'
* '334 Furman St, Brooklyn'
* '400th West 20th St., Suite 2N'
* '54 W 39th St New York, NY'
* 'E. 4th Street'
* 'Highline'

First all streets were changed to title case apart from some exceptions that should remain completely capitalized, like 'FDR Drive', and those like 'of' and 'the' that should be lowercase when not appearing as the first word.  After fully auditing, some of the streets were found to be completely invalid, like 'Track' and 'Washington Square Village'.  Values of None were emitted for these streets, and so were effectively removed from the data.  At this point there was a collection of streets that could not be corrected programmatically, and so were updated using a dictionary mapping.  These included '85th West Street' being changed to 'West 85th Street' and '29th' to '29th Street'.

The rest of the cities, if they were not in the previously mentioned mapping, were corrected mostly using regular expressions.  If the city, like 'New York' or 'Brooklyn' was present, that was removed from the street and was returned as the value for 'addr:city' tag key.  Then, commas were removed from the data set.  Regular expressions and a mapping were used to replace abbreviated directions for North, South, East, and West with their longer counterpart.  If a floor, suite, zip code, or housenumber was present these were removed from the street and were assigned to the 'addr:floor', 'addr:suite', 'addr:postcode', or 'addr:housenumber' tag key, respectively.  Regular expressions were used to find numbered streets that didn't use numbers to represent them (e.g. Fourth as opposed to 4th).  These were then replaced using a mapping to their numbered equivalent.  Finally, the street type (e.g. Street, Road, Place) was extracted using regular expressions, and it was replaced using a mapping if it was an abbreviation.

```python
def update_street(s):
    s = title_case(s, ['fdr', 'un'], ['of', 'the'])

    if s in invalid_streets:
        return None
    if s in full_street_mapping:
        s = full_street_mapping[s]    
    
    s = remove_city(s)
    s = remove_commas(s)
    
    # Fix directional abbreviations
    s = re.sub('[\s]*W[\.]*[\s]+', ' West ', s).strip()
    s = re.sub('[\s]*E[\.]*[\s]+', ' East ', s).strip()
    s = re.sub('[\s]*S[\.]*$', ' South', s).strip()
    
    s = remove_floor(s)
    s = remove_suite(s)
    s = remove_postcode(s)
    s = remove_housenumber(s)
    s = update_non_number(s)
    
    st_type = get_street_type(s)
    if st_type in street_type_mapping:
        s = re.sub(st_type, street_type_mapping[st_type], s)
    return s.strip()
```

'State Street & Water Street' was the only value for 'addr:street' that included more than one street.  This was the address for SeaGlass Carousel in The Battery.  Since both Google Maps and their official site list their address using both streets, I decided to keep both.

In [7]:
%%sql
SELECT value AS 'Street', COUNT(*) AS Count 
FROM tags
WHERE key='addr:street'
GROUP BY value
ORDER BY Count DESC
LIMIT 10;

Done.


Street,Count
Broadway,616
2nd Avenue,482
3rd Avenue,392
8th Avenue,343
1st Avenue,326
5th Avenue,295
6th Avenue,291
Grand Street,260
The Bowery,248
Canal Street,242


It is unsurprising that these are the streets that appear most frequently in the dataset; besides that there are an abundance of landmarks on Broadway, it along with the Avenues are the longest streets in Manhattan (being much longer than their cross streets).

In [6]:
%%sql
SELECT value AS 'Street', COUNT(*) AS Count 
FROM tags
WHERE key='addr:street' and value LIKE '% Street'
GROUP BY value
ORDER BY Count DESC
LIMIT 10;

Done.


Street,Count
Grand Street,260
Canal Street,242
West 22nd Street,242
East 14th Street,241
Greenwich Street,230
Bleecker Street,223
East 10th Street,222
West 20th Street,217
Hudson Street,211
West 11th Street,209


MAKE BETTERThe top ten streets have very similar frequencies, most likely since these streets are all ********** in length.

In [31]:
%%sql
--Most popular streets in the Brooklyn part of map

SELECT value AS 'Street', COUNT(*) AS Count 
FROM tags
WHERE key='addr:street' and id in (SELECT id
                                   FROM tags
                                   WHERE key='addr:city' and value='Brooklyn')
GROUP BY value
ORDER BY Count DESC
LIMIT 10;

Done.


Street,Count
Henry Street,12
Willow Street,9
Water Street,6
Front Street,5
Columbia Heights,4
Clark Street,3
Hicks Street,3
Jay Street,3
Main Street,3
Old Fulton Street,3


### Cities

There was inconsistency with how cities were represented.  The only two acceptable cities are New York and Brooklyn.  The addr:city tags erroneously included the state, referred to the neighborhood as opposed to the city, and had capitalization inconsistencies.  All those referring to New York, Manhattan, and its neighborhoods were mapped to New York; there were no problems with those marked 'Brooklyn.'

Example errors in city:
* 'Manhattan NYC'
* 'NEW YORK CITY'
* 'New York City'
* 'New York, NY'
* 'Tribeca'
* 'York City'
* 'new york'

In [5]:
%%sql
SELECT value AS 'City', COUNT(*) AS Count 
FROM tags
WHERE key='addr:city'
GROUP BY value
ORDER BY Count DESC
LIMIT 10;

Done.


City,Count
New York,3316
Brooklyn,77


###  States
Almost all of the values for addr:state were 'NY' with only the errors being those listed below.

Errors in state:
* '10009'
* 'New York'
* 'New York State'
* 'ny'

Since 'NY' is the only valid value, a function is not required and all values for the key 'addr:state' were assigned to 'NY'.

### Countries
453 of the values for the key addr:country were 'US', with only five being 'USA'.  These were corrected with a simple mapping.

### Full Address
There were three appearances of the 'addr:full' key in the data.  This tag key is only meant to be used when the address cannot be stated using the structured address fields.

The first value: '20 Jay Street #842, Brooklyn, New York 11201' did not meet this criteria.  Furthermore, this address also included all of the structured address tags (e.g. 'addr:housenumber' = '20').  The only tag it was missing was for 'addr:suite', which I added to the addtl_tags dictionary, so that it may be assigned to this address' tags.

The next value, '42nd Street Between 7th Avenue and Broadway' is for the Times Square Ball.  This address did not include any of the structured address tags.  This full address value was also unnecessary, since it has a well defined address at the top of One Times Square, also known as 1475 Broadway.  So, I added the appropriate values for housenumber, street, city, state, and postcode to addtl_tags.

The last value was for Bryant Park at 'Between 40th and 42nd Streets at 6th Avenue'.  I changed this to '6th Avenue between 40th and 42nd Streets', so that it had the between streets at the end of the address.  This address had all of the appropriate structured address tags.

### Phone Numbers
According to the OSM Wiki phones must be represented as a plus sign, country code, area code, local number all either separated by dashes or by spaces; I chose to use dashes.

Phone numbers were represented in a variety of ways, including with no spaces or dashes, no country code, with parentheses, using periods, extensions, or with multiple phone numbers.  Only the first phone number was emitted and no extensions were included in the updated numbers.

Example errors in phone numbers:
* '7188753700'
* '917 639 3089'
* '646) 205-7782'
* '2127547403#100,107,108,109'
* '212.229.2226'
* '1 (646) 598-0100'
* '001 646-6740001'
* '+12127777702'

Some phone numbers started correctly with the +1 country code for the United States, others only included the 1, others started with 001 (which is the exit code for most countries and how the +1 is dialed on phones that do not have the plus sign capability), and others did not include any part of the country code.

Values for the 'phone', 'contact:phone', 'phone:emergency', 'contact:fax', 'Phone', and 'phone_1' tags were updated using the code below.  In addition, the tag keys for 'Phone' and 'phone_1' were corrected to 'phone'.


```python
def is_valid_phone(phone):
    r = re.compile(r'^\+1-\d{3}-\d{3}-\d{4}$')
    if r.match(phone) is not None:        
        return True
    else:
        return False

# Use this to clean all phone number tag fields:
# contact:phone, Phone, phone_1, phone:emergency, fax, contact:fax
def update_phone(phone):
    """Returns the cleaned version of the phone number, and only returns
    the first phone number listed"""
    # replace multiple spaces with one space
    phone = re.sub('\s+', '', phone).strip()
    for ch in ['(',')','.','-']:
        phone = phone.replace(ch, '')
    
    if phone.startswith('001'):
        phone = '+' + phone[2:]
    if not phone.startswith('+'):
        phone = '+' + phone
    if phone[1] != '1':
        phone = phone[0] + '1' + phone[1:]
    
    phone = phone[0:2] + '-' + phone[2:5] + '-' + phone[5:8] + '-' + phone[8:]
    
    # Do not return second phone numbers or extensions
    phone = phone[0:15]
    
    if is_valid_phone(phone):
        return phone
    else:
        return None

```


In [6]:
%%sql
SELECT value AS 'Phone Number', COUNT(*) AS Count 
FROM tags
WHERE tags.key in ('phone', 'contact:phone', 'phone:emergency',
                   'contact:fax', 'Phone', 'phone_1', 'fax')
GROUP BY value
ORDER BY Count DESC
LIMIT 5;

Done.


Phone Number,Count
+1-844-359-2523,4
+1-212-533-4575,3
+1-212-219-0666,2
+1-212-227-3400,2
+1-212-233-8338,2


None of the phone numbers occurred more than 4 times in the data set and 913 of the phone numbers out of 932 only occurred once.

In [7]:
%%sql
CREATE VIEW phone_count_again AS
SELECT phone_counts.Count, COUNT(*) AS Frequency
FROM (SELECT COUNT(*) AS Count 
      FROM tags
      WHERE tags.key in ('phone', 'contact:phone', 'phone:emergency',
                   'contact:fax', 'Phone', 'phone_1', 'fax')
      GROUP BY value
      ORDER BY Count DESC) phone_counts
GROUP BY Count;

(sqlite3.OperationalError) table phone_count_again already exists [SQL: "CREATE VIEW phone_count_again AS\nSELECT phone_counts.Count, COUNT(*) AS Frequency\nFROM (SELECT COUNT(*) as Count \n      FROM tags\n      WHERE tags.key in ('phone', 'contact:phone', 'phone:emergency',\n                   'contact:fax', 'Phone', 'phone_1', 'fax')\n      GROUP BY value\n      ORDER BY Count DESC) phone_counts\nGROUP BY Count;"]


In [10]:
%%sql
SELECT *
FROM phone_count_again
LIMIT 10;

Done.


Count,Frequency
1,913
2,17
3,1
4,1


In [11]:
%%sql
SELECT pc1.Count, 100*pc1.Frequency/(SELECT SUM(Frequency) FROM phone_count_again) 'Percent of Total'
FROM phone_count_again pc1, phone_count_again pc2
WHERE pc1.Count = pc2.Count
GROUP BY pc1.Count

Done.


Count,Percent of Total
1,97
2,1
3,0
4,0


### Buildings
The values for the tag key 'building' were compared to the list of valid values assembled using BeautifulSoup from the OSM Wiki.  The value of 'publib' was a simple typo and was mapped to 'public'.  Some of the values were not valid and could not be mapped to logical alternatives; these were: 'no', 'tower', and 'works'.  The other values needed to have their key changed, as they were incorrectly labeled as buildings, and possibly their value changed as well.  For instance, 'college' and 'courthouse' should have been assigned to the key 'amenity', as should have 'convent', but its value needed to be adjusted to 'monastery'.

In [4]:
%%sql
SELECT value AS 'Building', COUNT(*) AS Count 
FROM tags
WHERE key='building'
GROUP BY value
ORDER BY Count DESC
LIMIT 10;

Done.


Building,Count
yes,18687
apartments,421
commercial,73
church,67
school,44
hotel,31
office,30
residential,19
roof,19
construction,14


The vast majority of building tags were only labeled with a 'yes' value.

### Amenities
The values for the tag key 'amenity' were cleaned in a similar fashion to those for 'building'.  Only one value only required a mapping of its value ('training' to 'school').  All others required a mapping of their key and/or value.  Fo instance, 'gym' needed to be changed to 'fitness_centre', but should have been assigned to the key 'leisure'.

In [5]:
%%sql
SELECT value AS 'Amenity', COUNT(*) AS Count 
FROM tags
WHERE key='amenity'
GROUP BY value
ORDER BY Count DESC
LIMIT 10;

Done.


Amenity,Count
bicycle_parking,1961
restaurant,1011
cafe,390
fast_food,277
bicycle_rental,231
bar,192
parking,181
bank,155
embassy,147
bench,140


### Shop
The values for the tag key 'shop' were cleaned in a similar fashion to those for 'building' and 'amenity'.  Four values requied a mapping of their key (e.g. 'car_rental' should have been an 'amenity'), and the rest of those needed to have their value updated (e.g. 'beer' to 'alcohol' and 'mattress' to 'bed'.  One of the values of 'comics', needed to be changed to a value of 'books', but also be given an additional tag with key 'books' and value 'comics'.

### Cuisine

There was a lot of inconsistencies in how values for the key 'cuisine' were represented.  Some values also erroneously included spaces (e.g. 'wine bar', while others were written as though they would have spaces, but with the spaces replaced with underscores ( 'american,_grilled_cheese,_tater_tots').    
 'italian coffee, breads, lite meals',
 
There were many values that had many different cuisines separated by commas, slashes, and semicolons.  

I compiled a list of the acceptable cuisines 


There was also one tag with the key 'cuisine_1' which I reassigned to 'cuisine'.

### Delivery

In [9]:
%%sql
SELECT value AS 'Delivery', COUNT(*) AS Count 
FROM tags
WHERE key='delivery'
GROUP BY value
ORDER BY Count DESC

Done.


Delivery,Count
yes,48
no,37
